# QONNX - Transformation passes
--------------------------------------
In this notebook the idea behind transformation passes in QONNX will be explained and with the help of an example the procedure of a transformation will be shown.

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [1]:
import inspect
import netron
import os
from IPython.display import IFrame
import urllib.request


def showSrc(what):
    print("".join(inspect.getsourcelines(what)[0]))


def showInNetron(model_filename: str, localhost_url: str = None, port: int = None):
    """Shows a ONNX model file in the Jupyter Notebook using Netron.

    :param model_filename: The path to the ONNX model file.
    :type model_filename: str

    :param localhost_url: The IP address used by the Jupyter IFrame to show the model.
     Defaults to localhost.
    :type localhost_url: str, optional

    :param port: The port number used by Netron and the Jupyter IFrame to show
     the ONNX model.  Defaults to 8081.
    :type port: int, optional

    :return: The IFrame displaying the ONNX model.
    :rtype: IPython.lib.display.IFrame
    """
    try:
        port = port or int(os.getenv("NETRON_PORT", default="8081"))
    except ValueError:
        port = 8081
    localhost_url = localhost_url or os.getenv("LOCALHOST_URL", default="localhost")
    netron.start(model_filename, address=("0.0.0.0", port), browse=False)
    return IFrame(src=f"http://{localhost_url}:{port}/", width="100%", height=400)

def download_model_from_zoo():
    qonnx_url="https://github.com/fastmachinelearning/QONNX_model_zoo/raw/main/models/MNIST/Brevitas_FINN_TFC/TFC/TFC_2W2A.onnx"
    dl_file="TFC_2W2A.onnx"
    urllib.request.urlretrieve(qonnx_url, dl_file)

## General Information
-----------------------------
* changes (transforms) the given graph
* input: ModelWrapper
* returns the changed model (ModelWrapper) and flag `model_was_changed`

Transformation passes have a base class and must inherit from that. Transformations are meant to be applied using .transform function from the ModelWrapper. This function makes a deep copy of the input model by default. The next cell shows .transform of ModelWrapper.


### .transform() from ModelWrapper

In [2]:
from qonnx.core.modelwrapper import ModelWrapper
showSrc(ModelWrapper.transform)

    def transform(self, transformation, make_deepcopy=True, cleanup=True):
        """Applies given Transformation repeatedly until no more changes can be made
        and returns a transformed ModelWrapper instance.

        - make_deepcopy : operates on a new (deep)copy of model.
        - cleanup : execute cleanup transformations before returning
        """
        transformed_model = self
        if make_deepcopy:
            transformed_model = copy.deepcopy(self)
        if self.fix_float64:
            (transformed_model, model_was_changed) = DoubleToSingleFloat().apply(transformed_model)
        model_was_changed = True
        while model_was_changed:
            (transformed_model, model_was_changed) = transformation.apply(transformed_model)
        if cleanup:
            transformed_model.cleanup()
        return transformed_model



When the function is called, the model, the name of the transformation and, if required, the flag make_deepcopy are passed. It is also possible not to make a copy of the model. In this case `make_deepcopy` must be set to False. Then the branch `if make_deepcopy:` would not be taken and no copy of the model would be made. 

The unchanged model is first passed to the variable `transformed_model` to pass this variable on to the transformation later. 

`model_was_changed` indicates whether the transformation needs to be applied more then once. Because it needs to be applied at least one time `model_was_changed` is first set to True and then depending on the return values of the transformation function the transformation can be applied more then once. 

**Important**: Due to the structure of this function, `model_was_changed` must be set to False at some point. Otherwise the loop is infinite.
    

Each new transformation must correspond to the scheme of the base class and contain at least the function `apply(model)`, which returns the changed model and a bool value for `model_was_changed`.

### Transformation Base Class     

In [3]:
from qonnx.transformation.base import Transformation

showSrc(Transformation)

class Transformation(ABC):
    """Transformation class all transformations are based on. Contains only
    abstract method apply() every transformation has to fill."""

    def __init__(self):
        super().__init__()

    @abstractmethod
    def apply(self, model):
        pass



Base class is abstract class (`import ABC`) with only one abstract method (`apply()`) which gets the model as input. To show what a transformation should look like, the following example is taken from a general-purpose QONNX transformation.

## Example - ConvertSubToAdd
-----------------------------
The transformation replaces all subtraction nodes in a model with addition nodes with appropriate sign. For that an onnx model is loaded which contains one subtraction node. 
    
Netron is used to visualize the result before and after.

In [5]:
download_model_from_zoo()
import onnx
onnx_model = onnx.load('TFC_2W2A.onnx')
from qonnx.core.modelwrapper import ModelWrapper
onnx_model = ModelWrapper(onnx_model)

In [6]:
showInNetron("TFC_2W2A.onnx")

Serving 'TFC_2W2A.onnx' at http://0.0.0.0:8081


In [7]:
from qonnx.transformation.base import Transformation

class ConvertSubToAdd(Transformation):
    def apply(self, model):
        graph = model.graph
        for n in graph.node:
            if n.op_type == "Sub":
                A = model.get_initializer(n.input[1])
                if A is not None:
                    n.op_type = "Add"
                    model.set_initializer(n.input[1], -A)
        return (model, False)

First the transformation class must be imported. Then a class can be created for the new transformation, which is derived from the base class. In this case the transformation has only the `apply()` function. 

All nodes are checked by first extracting the graph from the model and then iterating over the node list. With the help of .op_type the operation type of the node can be checked, if the node is a subtraction node the condition `if n.op_type == "Sub"` is true. It may be that the subtraction input of the node has no value, this is checked with `model.get_initializer(n.input[1])`. 
    
    
**Important:** As with ONNX, QONNX always assumes a certain order of inputs, this is especially important if you want to create additional custom operation nodes.

When the input is initialized, the operation type of the node is converted to `"Add"`, this can simply be done by using the equal sign. Then the sign of the initial value must be changed. For this the ModelWrapper function `.set_initializer` can be used.

At the end the changed model is returned and `model_was_changed` is set to False, because the transformation has to be executed only once.



In [8]:
onnx_model_transformed = onnx_model.transform(ConvertSubToAdd())
onnx_model_transformed.save('/tmp/LFCW1A1_changed.onnx')

In [9]:
showInNetron('/tmp/LFCW1A1_changed.onnx')

Stopping http://0.0.0.0:8081
Serving '/tmp/LFCW1A1_changed.onnx' at http://0.0.0.0:8081


## Parallel Transformation
---------------------------------
Some of the transformations in QONNX can be performed in parallel on individual nodes to make the transformations run quicker. The following `NodeLocalTransformation` is required for this:

In [10]:
from qonnx.transformation.base import NodeLocalTransformation

showSrc(NodeLocalTransformation)

class NodeLocalTransformation(Transformation):
    """
    Parent class for transformations, which can be executed locally to one node
    by accessing and modifying the attributes of only that node.
    This class can then automatically parallelize the transformation.
    Transformations sublcassing NodeLocalTransformation must implement the
    abstract method applyNodeLocal().

    To control the degree of parallelization, specify the num_workers argument
    in the constructor, using one of the following values:
    * None: use NUM_DEFAULT_WORKERS environment variable
    * 0: use all available CPU cores
    * (any other int>0): set number of parallel workers
    """

    def __init__(self, num_workers=None):
        super().__init__()
        if num_workers is None:
            self._num_workers = get_num_default_workers()
        else:
            self._num_workers = num_workers
        assert self._num_workers >= 0, "Number of workers must be nonnegative."
        if self._num_w

Transformations that are to be executed in parallel must have the method `applyNodeLocal()` implemented. Please note that the transformation is only executed on a single node, the parallel transformations do not have access to the entire model or tensors. Parallelization has the advantage that especially time-consuming transformations such as compilation can be executed more effectively. 

To control the degree of parallelization the argument `num_workers` can be specified. The environment variable `NUM_DEFAULT_WORKERS` controls this. You can set the number of workers manually to a specific value when calling a transformation that allows parallelization. If the value is set to 0, all available CPU cores are used.